# Acceso a Google Earth Engine y Descarga de Datos Recortados
En este notebook aprenderás a:
1. Autenticarte en Google Earth Engine (GEE).
2. Cargar datos de Maximum Water Extent.
3. Visualizar los datos en un mapa interactivo usando Leaflet.
4. Seleccionar un área y descargar los datos recortados en formato GeoTIFF.

---

In [ ]:
# Instalación de bibliotecas necesarias
!pip install streamlit leafmap earthengine-api geopandas

## Autenticación en Google Earth Engine
Para acceder a los datos, debes autenticarte en GEE utilizando tu cuenta de Google.
El siguiente bloque solicita el usuario y la contraseña para iniciar sesión.

In [ ]:
import ee

# 🔄 REEMPLAZAR estas credenciales con las tuyas propias
# Ejemplo: usuario = "tu_correo@ejemplo.com" y contrasena = "tu_contraseña_secreta"
usuario = "tu_correo@ejemplo.com"
contrasena = "tu_contraseña_secreta"

# Autenticación en Google Earth Engine
if usuario and contrasena:
    print("🔑 Credenciales ingresadas. Intentando autenticar...")
    try:
        # 🟢 Esta línea abrirá una ventana para autenticar si es necesario
        ee.Authenticate()  
        
        # 🟢 Reemplaza 'ee-tu_proyecto' con el nombre de tu proyecto en GEE si aplica
        ee.Initialize(project='ee-tu_proyecto')
        
        print("✅ Autenticación exitosa con Google Earth Engine.")
    except Exception as e:
        print(f"❌ Error de autenticación: {str(e)}")
else:
    print("⚠️ Credenciales no válidas. Asegúrate de reemplazar las credenciales dummy.")

## Cargar y visualizar datos de Global Surface Water
Este bloque carga los datos de GEE y los muestra en un mapa interactivo utilizando Leaflet, dentro de cada uno de ellos puedes dibujar un poligono de no mucha extensión y obtener un enlace para la descarga de los datos en formato .tif

In [ ]:
from ipyleaflet import Map, TileLayer, DrawControl, basemaps, LegendControl
import ee
import json
from IPython.display import display

# Función para agregar leyenda personalizada
def agregar_leyenda(mapa, title, legend_dict):
    legend = LegendControl(
        {k: v for k, v in legend_dict.items()},
        name=title,
        position="bottomright"
    )
    mapa.add_control(legend)


## 🌊 Water Occurrence
El dataset **Water Occurrence** muestra las áreas donde ha existido agua superficial entre **1984 y 2021**. Proporciona información sobre la **frecuencia** con la que el agua estuvo presente en la superficie, capturando tanto la **variabilidad intraanual** como la **interanual**.

- **Agua permanente (100%):** Áreas con agua constante durante todo el período.
- **Agua intermitente (1%-99%):** Áreas donde el agua estuvo presente de forma variable.
- **Sin agua:** Regiones sin detección de agua.
- **Sin datos:** Áreas sin cobertura de datos.

Este producto es útil para analizar la **dinámica a largo plazo** de los cuerpos de agua y los efectos del **cambio climático** y las **actividades humanas** en los recursos hídricos.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('occurrence')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/occurrence/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Water Occurrence", {
    "Agua permanente (100%)": "#0000FF",
    "Agua intermitente (1%-99%)": "#FF0000",
    "Sin agua": "#FFFFFF",
    "Sin datos": "#CCCCCC"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar los datos recortados a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })

    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)

## 📊 Occurrence Change Intensity
El dataset **Occurrence Change Intensity** analiza los cambios en la ocurrencia de agua superficial entre **1984 y 2021**.

- **Aumento de agua (100%):** Regiones donde la presencia de agua ha incrementado significativamente.
- **Disminución de agua (-100%):** Áreas donde la presencia de agua ha disminuido.
- **Sin cambios:** Regiones estables sin variaciones significativas.
- **Sin datos:** Áreas sin cobertura de datos.

Este producto ayuda a identificar tendencias y patrones en la disponibilidad de agua a lo largo del tiempo, crucial para la **gestión sostenible** de los recursos hídricos.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('change')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/change/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Change Intensity", {
    "Aumento de agua (100%)": "#00FF00",
    "Disminucion de agua (-100%)": "#FF0000",
    "Sin cambios": "#000000",
    "Sin datos": "#CCCCCC",
    "No se puede calcular": "#888888"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })
    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)


## 🌀 Seasonality
El dataset **Seasonality** muestra la **estacionalidad del agua superficial** indicando cuántos meses por año un área estuvo cubierta por agua entre **1984 y 2021**.

- **12 meses:** Agua permanente todo el año.
- **1 mes:** Agua estacional (solo presente algunos meses).
- **Sin agua:** Regiones sin presencia de agua durante el período.
- **Sin datos:** Áreas sin cobertura de datos.

Es útil para entender la **variabilidad estacional** de los cuerpos de agua y planificar la **gestión de cuencas hidrográficas**.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('seasonality')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/seasonality/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Seasonality", {
    "12 meses (Agua permanente)": "#0000AA",
    "1 mes (Agua estacional)": "#99D9EA",
    "Sin agua": "#FFFFFF",
    "Sin datos": "#CCCCCC"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })
    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)


## 🔄 Recurrence
El dataset **Recurrence** mide la **frecuencia con la que el agua reaparece** en el mismo lugar año tras año entre **1984 y 2021**.

- **Alta recurrencia (100%):** Áreas con aparición constante de agua.
- **Baja recurrencia (1%):** Regiones donde el agua aparece esporádicamente.
- **Sin agua:** Áreas sin detección de agua.
- **Sin datos:** Regiones sin cobertura de datos.

Este producto permite identificar áreas con ciclos de agua bien definidos y planificar el **uso de recursos hídricos** en función de su predictibilidad.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('recurrence')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/recurrence/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Recurrence", {
    "Alta recurrencia (100%)": "#99D9EA",
    "Baja recurrencia (1%)": "#FF7F27",
    "Sin agua": "#FFFFFF",
    "Sin datos": "#CCCCCC"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })
    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)


## 🔄 Transitions
El dataset **Transitions** analiza los **cambios en los estados de agua superficial** entre **1984 y 2021**.

- **Agua permanente:** Regiones que mantuvieron agua constante.
- **Nueva agua permanente:** Áreas donde apareció agua de forma estable.
- **Pérdida de agua permanente:** Regiones que dejaron de tener agua estable.
- **Estacional a permanente:** Cambios de agua estacional a permanente.
- **Ephemeral:** Agua que apareció temporalmente.

Este producto es clave para comprender la **dinámica de los cuerpos de agua** y sus **transiciones** debido a fenómenos naturales y actividades humanas.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('transitions')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/transitions/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Transitions", {
    "Agua permanente": "#0000FF",
    "Nueva agua permanente": "#22B14C",
    "Perdida de agua permanente": "#D1102D",
    "Agua estacional": "#99D9EA",
    "Nueva agua estacional": "#B5E61D",
    "Perdida de agua estacional": "#E6A1AA",
    "Estacional a permanente": "#FF7F27",
    "Permanente a estacional": "#FFC90E",
    "Ephemeral permanente": "#7F7F7F",
    "Ephemeral estacional": "#C3C3C3",
    "Sin agua": "#FFFFFF",
    "Sin datos": "#CCCCCC"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })
    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)


## 📏 Maximum Water Extent
El dataset **Maximum Water Extent** muestra el **área máxima cubierta por agua superficial** entre **1984 y 2021**.

- **Agua detectada:** Áreas que alguna vez estuvieron cubiertas por agua.
- **Sin agua:** Regiones donde no se detectó agua.
- **Sin datos:** Áreas sin cobertura de datos.

Este producto es esencial para mapear la **distribución espacial de los cuerpos de agua** y entender la **extensión máxima histórica** de inundaciones y lagos.


In [ ]:
dataset = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('max_extent')

m = Map(center=(9.144571, -74.634518), zoom=8, basemap=basemaps.CartoDB.Positron)
m.add_layer(TileLayer(url="https://storage.googleapis.com/global-surface-water/tiles2021/extent/{z}/{x}/{y}.png"))

agregar_leyenda(m, "Leyenda - Maximum Water Extent", {
    "Agua detectada": "#6666FF",
    "Sin agua": "#FFFFFF",
    "Sin datos": "#CCCCCC"
})

draw_control = DrawControl()
m.add_control(draw_control)
display(m)

def on_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates']
    geometry = ee.Geometry.Polygon(coords)
    clip = dataset.clip(geometry)

    # Exportar a GeoTIFF
    print("⏳ Generando enlace de descarga de GeoTIFF...")
    url = clip.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': geometry.toGeoJSONString(),
        'fileFormat': 'GeoTIFF'
    })
    print(f"✅ Enlace generado: [Descargar GeoTIFF]({url})")

draw_control.on_draw(on_draw)


## Conclusión
En este notebook aprendiste cómo autenticarte en Google Earth Engine, cargar y visualizar datos geoespaciales y descargar un área recortada en formato GeoTIFF.

Para más información, visita la [documentación oficial de Google Earth Engine](https://developers.google.com/earth-engine/).